In [38]:
model_file_path = "/home/gsoykan/Desktop/comp541/comp541_term_project/results/imagenet-resnet-152-dag.mat"
cat_img_url = "https://nextjournal.com/data/QmXNbi2LE7u6yBdBXaQ9E2zGb48FELg3TxjrLiPKBmdvZc?filename=Qat.jpg&content-type=image/jpeg"

"https://nextjournal.com/data/QmXNbi2LE7u6yBdBXaQ9E2zGb48FELg3TxjrLiPKBmdvZc?filename=Qat.jpg&content-type=image/jpeg"

In [2]:
# TODO: implement resnet
# https://nextjournal.com/mpd/image-classification-with-knet

In [14]:
using MAT, OffsetArrays, FFTViews, ArgParse, Images, Knet, ImageMagick

In [26]:
function get_params(params, atype)
    len = length(params["value"])
    ws, ms = [], []
    for k = 1:len
        name = params["name"][k]
        value = convert(Array{Float32}, params["value"][k])

        if endswith(name, "moments")
            push!(ms, reshape(value[:,1], (1,1,size(value,1),1)))
            push!(ms, reshape(value[:,2], (1,1,size(value,1),1)))
        elseif startswith(name, "bn")
            push!(ws, reshape(value, (1,1,length(value),1)))
        elseif startswith(name, "fc") && endswith(name, "filter")
            push!(ws, transpose(reshape(value,(size(value,3),size(value,4)))))
        elseif startswith(name, "conv") && endswith(name, "bias")
            push!(ws, reshape(value, (1,1,length(value),1)))
        else
            push!(ws, value)
        end
    end
    map(wi->convert(atype, wi), ws),
    map(mi->convert(atype, mi), ms)
end

get_params (generic function with 1 method)

In [27]:
# From vgg.jl
function data(img, averageImage)
    if occursin("://",img)
        @info "Downloading $img"
        img = download(img)
    end
    a0 = load(img)
    new_size = ntuple(i->div(size(a0,i)*224,minimum(size(a0))),2)
    a1 = Images.imresize(a0, new_size)
    i1 = div(size(a1,1)-224,2)
    j1 = div(size(a1,2)-224,2)
    b1 = a1[i1+1:i1+224,j1+1:j1+224]
    c1 = permutedims(channelview(b1), (3,2,1))
    d1 = convert(Array{Float32}, c1)
    e1 = reshape(d1[:,:,1:3], (224,224,3,1))
    f1 = (255 * e1 .- averageImage)
    g1 = permutedims(f1, [2,1,3,4])
end

data (generic function with 1 method)

In [16]:
# Batch Normalization Layer
# works both for convolutional and fully connected layers
# mode, 0=>train, 1=>test
function batchnorm(w, x, ms; mode=1, epsilon=1e-5)
    mu, sigma = nothing, nothing
    if mode == 0
        d = ndims(x) == 4 ? (1,2,4) : (2,)
        s = prod(size(x,d...))
        mu = sum(x,d) / s
        x0 = x .- mu
        x1 = x0 .* x0
        sigma = sqrt(epsilon + (sum(x1, d)) / s)
    elseif mode == 1
        mu = popfirst!(ms)
        sigma = popfirst!(ms)
    end

    # we need getval in backpropagation
    push!(ms, AutoGrad.value(mu), AutoGrad.value(sigma))
    xhat = (x.-mu) ./ sigma
    return w[1] .* xhat .+ w[2]
end

batchnorm (generic function with 1 method)

In [17]:
function reslayerx0(w,x,ms; padding=0, stride=1, mode=1)
    b  = conv4(w[1],x; padding=padding, stride=stride)
    bx = batchnorm(w[2:3],b,ms; mode=mode)
end

reslayerx0 (generic function with 1 method)

In [19]:
function reslayerx1(w,x,ms; padding=0, stride=1, mode=1)
    relu.(reslayerx0(w,x,ms; padding=padding, stride=stride, mode=mode))
end

reslayerx1 (generic function with 1 method)

In [20]:
function reslayerx2(w,x,ms; pads=[0,1,0], strides=[1,1,1], mode=1)
    ba = reslayerx1(w[1:3],x,ms; padding=pads[1], stride=strides[1], mode=mode)
    bb = reslayerx1(w[4:6],ba,ms; padding=pads[2], stride=strides[2], mode=mode)
    bc = reslayerx0(w[7:9],bb,ms; padding=pads[3], stride=strides[3], mode=mode)
end

reslayerx2 (generic function with 1 method)

In [21]:
function reslayerx3(w,x,ms; pads=[0,0,1,0], strides=[2,2,1,1], mode=1) # 12
    a = reslayerx0(w[1:3],x,ms; stride=strides[1], padding=pads[1], mode=mode)
    b = reslayerx2(w[4:12],x,ms; strides=strides[2:4], pads=pads[2:4], mode=mode)
    relu.(a .+ b)
end

reslayerx3 (generic function with 1 method)

In [22]:
function reslayerx4(w,x,ms; pads=[0,1,0], strides=[1,1,1], mode=1)
    relu.(x .+ reslayerx2(w,x,ms; pads=pads, strides=strides, mode=mode))
end

reslayerx4 (generic function with 1 method)

In [23]:
function reslayerx5(w,x,ms; strides=[2,2,1,1], mode=1)
    x = reslayerx3(w[1:12],x,ms; strides=strides, mode=mode)
    for k = 13:9:length(w)
        x = reslayerx4(w[k:k+8],x,ms; mode=mode)
    end
    return x
end

reslayerx5 (generic function with 1 method)

In [24]:
# mode, 0=>train, 1=>test
function resnet152(w,x,ms; mode=1)
    # layer 1
    conv1 = reslayerx1(w[1:3],x,ms; padding=3, stride=2, mode=mode)
    pool1 = pool(conv1; window=3, stride=2)

    # layer 2,3,4,5
    r2 = reslayerx5(w[4:33], pool1, ms; strides=[1,1,1,1], mode=mode)
    r3 = reslayerx5(w[34:108], r2, ms; mode=mode)
    r4 = reslayerx5(w[109:435], r3, ms; mode=mode)
    r5 = reslayerx5(w[436:465], r4, ms; mode=mode)

    # fully connected layer
    pool5  = pool(r5; stride=1, window=7, mode=2)
    fc1000 = w[466] * mat(pool5) .+ w[467]
end

resnet152 (generic function with 1 method)

In [39]:
o = Dict(
  :atype => KnetArray{Float32},
  :model => model_file_path,
  :image => cat_img_url,
  :top   => 10
)

Dict{Symbol,Any} with 4 entries:
  :atype => KnetArray{Float32,N} where N
  :top   => 10
  :image => "https://nextjournal.com/data/QmXNbi2LE7u6yBdBXaQ9E2zGb48FELg3TxjrL…
  :model => "/home/gsoykan/Desktop/comp541/comp541_term_project/results/imagene…

In [40]:
function predict(o)
	@info "Reading $(o[:model])"
	model = matread(abspath(o[:model]))
	avgimg = model["meta"]["normalization"]["averageImage"]
	avgimg = convert(Array{Float32}, avgimg)
	description = model["meta"]["classes"]["description"]
	w, ms = get_params(model["params"], o[:atype])

	@info "Reading $(o[:image])"
	img = data(o[:image], avgimg)
	img = convert(o[:atype], img)

	@info "Classifying."
	@time y1 = resnet152(w,img,ms)
  
  return y1, description
end

predict (generic function with 1 method)

In [41]:
y1, description = predict(o)

┌ Info: Reading /home/gsoykan/Desktop/comp541/comp541_term_project/results/imagenet-resnet-152-dag.mat
└ @ Main In[40]:2
┌ Info: Reading https://nextjournal.com/data/QmXNbi2LE7u6yBdBXaQ9E2zGb48FELg3TxjrLiPKBmdvZc?filename=Qat.jpg&content-type=image/jpeg
└ @ Main In[40]:9
┌ Info: Downloading https://nextjournal.com/data/QmXNbi2LE7u6yBdBXaQ9E2zGb48FELg3TxjrLiPKBmdvZc?filename=Qat.jpg&content-type=image/jpeg
└ @ Main In[27]:4
--2020-11-26 01:05:42--  https://nextjournal.com/data/QmXNbi2LE7u6yBdBXaQ9E2zGb48FELg3TxjrLiPKBmdvZc?filename=Qat.jpg&content-type=image/jpeg
Resolving nextjournal.com (nextjournal.com)... 104.155.21.143
Connecting to nextjournal.com (nextjournal.com)|104.155.21.143|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 107863 (105K) [image/jpeg]
Saving to: ‘/tmp/jl_GjsrSM’

     0K .......... .......... .......... .......... .......... 47%  435K 0s
    50K .......... .......... .......... .......... .......... 94%  920K 0s
   100K .....           

  0.052981 seconds (31.10 k allocations: 1.343 MiB)


(K32(1000,1)[-1.847644⋯], Any["tench, Tinca tinca" "goldfish, Carassius auratus" … "ear, spike, capitulum" "toilet tissue, toilet paper, bathroom tissue"])

In [35]:
z1 = vec(Array(y1))
s1 = sortperm(z1,rev=true)
p1 = exp.(logp(z1))

1000-element Array{Float32,1}:
 7.434242f-6
 7.9623715f-6
 1.4927994f-7
 6.341796f-8
 6.8408036f-7
 9.860025f-5
 4.4320145f-6
 1.476855f-5
 0.0009105815
 3.427741f-5
 9.599371f-6
 1.4747186f-5
 1.6379536f-5
 ⋮
 1.1692983f-5
 7.740901f-8
 1.8692223f-6
 5.7157f-7
 1.0856252f-6
 2.5791576f-6
 1.1585938f-6
 7.935305f-7
 1.1721868f-5
 7.2542894f-6
 1.1682383f-5
 1.5206274f-5

In [36]:
using Printf

for ind in s1[1:o[:top]]
  print("$(description[ind]): $(@sprintf("%.2f",p1[ind]*100))%\n")
end

gibbon, Hylobates lar: 51.15%
Pomeranian: 9.16%
marmoset: 7.37%
langur: 7.00%
capuchin, ringtail, Cebus capucinus: 4.09%
Pembroke, Pembroke Welsh corgi: 3.39%
ringlet, ringlet butterfly: 3.38%
hamster: 2.25%
macaque: 1.89%
Siamese cat, Siamese: 1.18%
